In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
cd 'drive/My Drive/final/'

/content/drive/.shortcut-targets-by-id/1LlyPSYhxPjZ3mmfbs80U9Ni-AoklGoNU/final


In [0]:
# Uncomment only if you do not already have all of the dependancies downloaded. 

#!git clone https://github.com/openai/train-procgen.git
!pip install https://github.com/openai/baselines/archive/9ee399f5b20cd70ac0a871927a6cf043b478193f.zip
!pip install -e train-procgen

#!git clone https://github.com/openai/baselines.git
!pip install baselines

In [0]:
cd 'train-procgen'

/content/drive/My Drive/Colab Notebooks/final/train-procgen


In [0]:
!pip install procgen
!pip install mpi4py
!pip install gym

In [0]:
%load_ext autoreload
%autoreload 2
!pip install --upgrade tensorflow-gpu==1.15.0

In [0]:
"""
!!!WARNING!!! --> If using Jupyter Notebook, Google Colab, or any other iPython notebook client, you must restart the kernel (runtime)
before running this cell again. The implementation set forth by OpenAI does not close the tensorflow session and clear the computation graph,
thus when you try to call this function again, the variables have already been allocated and tensorflow gets mad. I have tried going into the source
code and clearing the graph at the end of training, but I still encountered this error. The most reliable and least painful fix is to just restart the
kernel if you plan on training again.

Taken from https://github.com/openai/train-procgen.
Default is to use ppo2. Here we modify to use dqn instead of ppo.

TRAIN-PROCGEN using DEEP Q-NETWORK
implementation of DQN courtesy of OpenAI: https://github.com/openai/baselines/blob/master/baselines/deepq/deepq.py 
Hyperparameters: see defaults in cell above
"""

import tensorflow as tf
import gym
from baselines.ppo2 import ppo2
from baselines.deepq import deepq
from baselines.common.models import build_impala_cnn
from baselines.common.mpi_util import setup_mpi_gpus
from procgen import ProcgenEnv
from baselines.common.vec_env import (
    VecExtractDictObs,
    VecMonitor,
    VecFrameStack,
    VecNormalize
)
from baselines import logger
from mpi4py import MPI
import argparse

LOG_DIR = 'log/procgen'

env_name='procgen:procgen-fruitbot-v0'
start_level=0
distribution_mode='easy'

total_timesteps=50000
num_levels=100
buffer_size=10000
test_worker_interval = 0

gamma=0.999
learning_rate=5e-4

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

is_test_worker = False

if test_worker_interval > 0:
    is_test_worker = comm.Get_rank() % test_worker_interval == (test_worker_interval - 1)

mpi_rank_weight = 0 if is_test_worker else 1
num_levels = 0 if is_test_worker else num_levels

log_comm = comm.Split(1 if is_test_worker else 0, 0)
format_strs = ['csv', 'stdout'] if log_comm.Get_rank() == 0 else []
logger.configure(dir=LOG_DIR, format_strs=format_strs)

logger.info("creating environment")

env = gym.make(env_name, start_level=start_level, num_levels=num_levels, distribution_mode=distribution_mode)

logger.info("creating tf session")
setup_mpi_gpus()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True #pylint: disable=E1101
sess = tf.Session(config=config)
sess.__enter__()

conv_fn = lambda x: build_impala_cnn(x, depths=[16,32,32], emb_size=256)

logger.info("training")
model = deepq.learn(
    env,
    network=conv_fn,
    seed=420,
    dueling=False,
    lr=5e-4,
    checkpoint_path="fruitbot_checkpoint",
    total_timesteps=2000000,
    print_freq=100,
    buffer_size=50000,
    exploration_fraction=0.2,
    exploration_final_eps=0.05,
    learning_starts=100000,
    target_network_update_freq=1000,
    gamma=0.99999,
    prioritized_replay=True,
    param_noise=False
)


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Logging to log/procgen
creating environment
creating tf session
training





Instructions for updating:
Use `tf.cast` instead.

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.Dense instead.








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


--------------------------------------
| % time spent exploring  | 97       |
| episodes                | 100      |
| mean 100 episode reward | -2.6     |
| steps                   | 1.02e+04 |
--------------------------------------
--------------------------------------
| % time spent exploring  | 95       |
| episodes                | 200      |
| mean 100 episode reward | -2.2     |
| steps                   | 1.99e+04 |
--------------------------------------
--------------------------------------
| % time spent exploring  | 92       |
| episodes                | 300      |
| mean 100 episode reward | -2.4     |
| steps                   | 2.95e+04 |
--------------------------------------
--------------------------------------
| % time spent exploring  | 90       |
| episodes                | 400      |
| mean 100 episode reward | -2.2     |
| steps                   | 3.95e+04 |
--------------------------------------
--------------------------------------
| % time spent exploring 

In [0]:
conv_fn = lambda x: build_impala_cnn(x, depths=[16,32,32], emb_size=256)
model = deepq.learn(gym.make('procgen:procgen-fruitbot-v0'), 
                    load_path='drive/My Drive/final/fruitbot_checkpoint',
                    network=conv_fn,
                    total_timesteps=1000)
while True:
        obs, done = env.reset(), False
        episode_rew = 0
        while not done:
            env.render()
            obs, rew, done, _ = env.step(model(obs[None])[0])
            episode_rew += rew
        print("Episode reward", episode_rew)

In [0]:
!python -m train_procgen.train --env_name fruitbot # Uses the PPO2 default values in train-procgen. This is the baseline.